In [2]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from typing import List, Literal, Optional, TypedDict
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langgraph.graph import StateGraph, END
import re
import pandas as pd

#emdeding_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emdeding_id = "jhgan/ko-sbert-sts"
qa_model_id = "monologg/koelectra-base-v3-finetuned-korquad"
#gen_model_id = "EleutherAI/polyglot-ko-1.3b"
#gen_model_id = "skt/kogpt2-base-v2"
gen_model_id = "beomi/gemma-ko-2b"

tokenizer_opt = {
    "max_length": 512, 
    "truncation": True,
    "do_sample": True,
    "device": -1
}

# 1. 문서 로드 및 VectorDB 저장
filepath = "./NewBie_개발환경가이드.docx"
persist_directory = "./chromaDB_Chatbot"
#filepath = "./data/data.csv"
#persist_directory = "./ChromaDB_Chatbot_test"


def load_embedding_model():
    return HuggingFaceEmbeddings(model_name=emdeding_id)

def load_QA_model(tokenizer_opt):
    qa_pipeline = pipeline(
        task="question-answering",
        model=qa_model_id,
        tokenizer_kwargs=tokenizer_opt,
        device=-1 
    )
    return qa_pipeline

def load_Gen_model():
    tokenizer = AutoTokenizer.from_pretrained(gen_model_id)
    model = AutoModelForCausalLM.from_pretrained(gen_model_id)
    return tokenizer, model


# Word 문서 로드 함수
def load_word_documents(filepath: str):
    loader = UnstructuredWordDocumentLoader(filepath)
    docs = loader.load()
    all_text = "\n".join([doc.page_content for doc in docs])  # doc.page_content로 수정
    sentences = re.split(r'\.|\n', all_text)
    sentences = [s.strip() for s in sentences if s.strip()]
    print("문장 개수:", len(sentences))
    print("샘플 문장 10개:", sentences[:10])
    print("마지막 10문장:", sentences[-10:])
    return sentences


def load_csv_documents(filepath):
    df = pd.read_csv(filepath, encoding="utf-8")
    texts = df["text"].tolist()
    docs = [Document(page_content=text) for text in texts]
    return docs

# VectorDB 저장 함수
def save_vectorDB(docs, persist_directory):
    #embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    embedding_model = load_embedding_model()

    vector_db = Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=persist_directory
)
    return vector_db

# VectorDB에서 검색(retrieve) 함수
def load_vectorDB(persist_directory):
    #embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    embedding_model = load_embedding_model()
    vector_db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model
)
    return vector_db


In [3]:
def run_qa(question, vector_db, qa_model_id, tokenizer_opt):
    retriever = vector_db.as_retriever(search_kwargs={"k":5})
    relevant_docs = retriever.invoke(question)
    if len(relevant_docs) == 0:
        print("검색 결과가 없습니다.")
        return None, None
    # 여러 문서를 context로 합침 (길이 증가)
    best_context = "\n".join([doc.page_content for doc in relevant_docs[:5]])
    print("[검색된 컨텍스트]", best_context)
    # qa_pipeline = pipeline(
    #     task="question-answering",
    #     model=qa_model_id,
    #     tokenizer_kwargs=tokenizer_opt,
    #     device = -1
    # )
    qa_pipeline = load_QA_model(tokenizer_opt)
    result = qa_pipeline(question=question,context=best_context)
    print("[QA 답변]", result['answer'])
    return result, best_context

In [4]:
#docs = load_csv_documents(filepath)
sentences = load_word_documents(filepath)
#docs = [Document(page_content=s) for s in sentences]  # 각 문장을 Document로 변환
#save_vectorDB(docs, persist_directory)



문장 개수: 1905
샘플 문장 10개: ['Pull Request 과정', '1', 'Git action flow 의 이해', 'file Pre-merge (Essential 통합)', 'github/workflows/01_premerge_pull_request', 'yaml premerge test 주관하는 workflow a', 'Integration test', '/', 'github/workflows/shared_premerge_integration_test', 'yaml b']
마지막 10문장: ['커버리지 예시 (위의 workflow run중에 2번째)', '[AWIBOF-10103] add error id to log (#283) · poseidonos/pos-essential-orchestrator@2c322ac (github', 'com)', '클릭해서 가장 밑으로 스크롤하면 확인 가능', 'Trouble Shooting', '1', 'mockery 오류', '권한 문제', 'rsync등으로 코드만 미러링하여 해당 서버의 gopath에 필요한 패키지들이 없는 경우', 'go mod tidy 가 잘되면 문제없이 mockery 실행 가능']


In [25]:
vector_db = load_vectorDB(persist_directory)
question = "TDK 는 무엇의 약자인가?"
result, best_context = run_qa(question, vector_db, qa_model_id, tokenizer_opt)

[검색된 컨텍스트] TDK는 다음과 같은 특징을 가지고 있습니다:
TDK는 다음과 같은 특징을 가지고 있습니다:
TDK는 이러한 문제를 해결하기 위해 만들어졌습니다
TDK는 이러한 문제를 해결하기 위해 만들어졌습니다
Test Development Kit (TDK)


Device set to use cpu


[QA 답변] Test Development Kit


In [ ]:
from langchain_core.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.chat_history import InMemoryChatMessageHistory
import uuid


def remove_duplicate_sentences(text):
    sentences = text.split('\n')
    seen = set()
    result = []
    for s in sentences:
        s_strip = s.strip()
        if s_strip and s_strip not in seen:
            result.append(s_strip)
            seen.add(s_strip)
    return '\n'.join(result)

# 생성형 모델 준비

#gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_id)
#gen_model = AutoModelForCausalLM.from_pretrained(gen_model_id)
gen_tokenizer, gen_model = load_Gen_model()
gen_tokenizer.model_max_length = 1024
gen_pipeline = pipeline(
    task="text-generation",
    model=gen_model,
    tokenizer=gen_tokenizer,
    max_new_tokens=200,
    truncation=True,
    do_sample=True,
    temperature=0.1,
    top_p=0.8,
    repetition_penalty=1.2,
    device=-1
)
gen_llm = HuggingFacePipeline(pipeline=gen_pipeline)


# 프롬프트 템플릿 정의 (질문 반복 방지, 답변만 생성)
prompt = PromptTemplate.from_template("""
당신은 Newbie가 아닌 시니어 개발자입니다.                                     
아래 내용을 참고하여 자연스럽고 명확한 한국어로 두세 문장으로 정리된 답변만 출력하세요.

[참고 문서]
{context}

[정답]
{answer}

[최종 답변]
""")

chain = prompt | gen_llm

final_response = chain.invoke({
    "question": question,
    "context": best_context,
    "answer": result['answer']
})
final_response = remove_duplicate_sentences(final_response)
final_response = ". ".join(list(dict.fromkeys(final_response.split(". "))))
print(final_response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


당신은 Newbie가 아닌 시니어 개발자입니다.
아래 내용을 참고하여 자연스럽고 명확한 한국어로 두세 문장으로 정리된 답변만 출력하세요.
[참고 문서]
TDK는 다음과 같은 특징을 가지고 있습니다:
TDK는 이러한 문제를 해결하기 위해 만들어졌습니다
Test Development Kit (TDK)
[정답]
Test Development Kit
[최종 답변]
TDK는 다음과 같이 사용됩니다:
TDK는 다음과 같이 사용됩니다
